In [2]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-02-06 13:48:21--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   146MB/s    in 0.3s    

2020-02-06 13:48:22 (146 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-02-06 13:48:22--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Reusing existing connection to rajpurkar.github.io:443.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  --.-KB/s    in 0.02s   

2020-02-06 13:48:22 (258 MB/s) - ‘dev-v2.0.json’ saved [4370528/4370528]

FINISHED --

In [3]:
import os
import random
import timeit

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm_notebook

from transformers import (
    AdamW,
    # BERT
    BertConfig,
    BertForQuestionAnswering,
    BertTokenizer,
    # AUTO
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    # XLM
    XLMTokenizer, XLMForQuestionAnswering,
    # XLNet
    XLNetTokenizer, XLNetForQuestionAnswering,
    # DistilBERT
    DistilBertTokenizer, DistilBertForQuestionAnswering,


    get_linear_schedule_with_warmup,
    squad_convert_examples_to_features,
)

from transformers.data.metrics.squad_metrics import (
    compute_predictions_logits,
    squad_evaluate,
)
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor

In [0]:
# Set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [0]:
# Setup CUDA, GPU & distributed training
device = torch.device("cuda")

In [6]:
def load_model(name):

  config = BertConfig.from_pretrained('bert-base-cased')
  tokenizer = BertTokenizer.from_pretrained('bert-base-cased',do_lower_case=True,)
  

  if name == "BERT":
    config = BertConfig.from_pretrained('bert-base-cased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased',do_lower_case=True,)
    model = BertForQuestionAnswering.from_pretrained('bert-base-cased',config=config,)

  
  elif name == "XLNET":
    tokenizer =  XLNetTokenizer.from_pretrained('xlnet-base-cased')
    model = XLNetForQuestionAnswering.from_pretrained('xlnet-base-cased')

  elif name == "DistilBERT":
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

  return model, tokenizer


#the nn.module BertForQuestionAnswering has a single untrained layer qa_output: Linear(hidden_size,2) on top of the trained BERT-base.
#model = BertForQuestionAnswering.from_pretrained('bert-base-cased',config=config,)
#print(list(model.children())[1])


#model = torch.nn.Sequential(list(model.children())[0])


#model.lstm = torch.nn.LSTM(768, 768)



model, tokenizer = load_model("XLM")
model.to(device)

XLMForQuestionAnswering(
  (transformer): XLMModel(
    (position_embeddings): Embedding(512, 2048)
    (embeddings): Embedding(30145, 2048, padding_idx=2)
    (layer_norm_emb): LayerNorm((2048,), eps=1e-12, elementwise_affine=True)
    (attentions): ModuleList(
      (0): MultiHeadAttention(
        (q_lin): Linear(in_features=2048, out_features=2048, bias=True)
        (k_lin): Linear(in_features=2048, out_features=2048, bias=True)
        (v_lin): Linear(in_features=2048, out_features=2048, bias=True)
        (out_lin): Linear(in_features=2048, out_features=2048, bias=True)
      )
      (1): MultiHeadAttention(
        (q_lin): Linear(in_features=2048, out_features=2048, bias=True)
        (k_lin): Linear(in_features=2048, out_features=2048, bias=True)
        (v_lin): Linear(in_features=2048, out_features=2048, bias=True)
        (out_lin): Linear(in_features=2048, out_features=2048, bias=True)
      )
      (2): MultiHeadAttention(
        (q_lin): Linear(in_features=2048, out_fe

In [0]:
max_seq_length=384
def load_and_cache_examples(tokenizer, is_training=True):
    # Load data features from cache or dataset file
    cached_features_file = "cached_{}".format("train" if is_training else "dev")

    # Init features and dataset from cache if it exists
    if os.path.exists(cached_features_file):
        print("Loading features from cached file ", cached_features_file)
        features_and_dataset = torch.load(cached_features_file)
        features, dataset, examples = (
            features_and_dataset["features"],
            features_and_dataset["dataset"],
            features_and_dataset["examples"],
        )
    else:
        print("Creating features from dataset file")
        
        if is_training:
            examples = SquadV2Processor().get_train_examples('')
        else:
            examples = SquadV2Processor().get_dev_examples('')

        features, dataset = squad_convert_examples_to_features(
            examples,tokenizer,max_seq_length,
            doc_stride=128,
            max_query_length=64,
            is_training=is_training,
            return_dataset="pt")

        print("Saving features into cached file", cached_features_file)
        torch.save({"features": features, "dataset": dataset, "examples": examples}, cached_features_file)

    return dataset, examples, features

In [8]:
train_dataset = load_and_cache_examples(tokenizer, is_training=True)[0]

Creating features from dataset file


add example index and unique id: 100%|██████████| 130319/130319 [00:00<00:00, 830253.67it/s]


Saving features into cached file cached_train


In [11]:
# Training
""" Train the model """
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=64)
epochs = 1
t_total = len(train_dataloader) * epochs

# Prepare optimizer and schedule (linear warmup and decay)
optimizer = AdamW(model.parameters(), lr=3e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=t_total)

# Train!
print("***** Running training *****")
print("  Num examples = ", len(train_dataset))
print("  Total optimization steps = ", t_total)

global_step = 1
tr_loss = 0.0
model.zero_grad()

for epoch in range(epochs):
    print('Epoch:{}'.format(epoch+1))
    epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration", disable=False)
    for step, batch in enumerate(epoch_iterator):

        model.train()
        batch = tuple(t.to(device) for t in batch)

        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
            "start_positions": batch[3],
            "end_positions": batch[4],
        }

        outputs = model(**inputs)
        # model outputs are always tuple in transformers (see doc)
        loss = outputs[0]

        loss.backward()

        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()
        global_step += 1
        
        # Log metrics
        if global_step % 50 == 0:
            # Only evaluate when single GPU otherwise metrics may not average well
            print('Global step = {}, logging_loss = {}'.format(global_step,tr_loss))

print(" global_step = %s, average loss = %s", global_step, tr_loss / global_step)

***** Running training *****
  Num examples =  133386
  Total optimization steps =  2085
Epoch:1


RuntimeError: ignored

In [0]:
# Save the trained model and the tokenizer
output_dir = 'output/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model checkpoint to %s", output_dir)
# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model checkpoint to %s output/


('output/vocab.txt',
 'output/special_tokens_map.json',
 'output/added_tokens.json')

In [0]:
# Load directly the trained weights
model = BertForQuestionAnswering.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=True)
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [0]:
# Evaluate
dataset, examples, features = load_and_cache_examples(tokenizer, is_training=False)
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

# Eval!
print("***** Running evaluation *****")
print("  Num examples = ", len(dataset))

all_results = []
start_time = timeit.default_timer()

for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
        }

        example_indices = batch[3]

        outputs = model(**inputs)

    for i, example_index in enumerate(example_indices):
        eval_feature = features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        output = [output[i].detach().cpu().tolist() for output in outputs]
        start_logits, end_logits = output
        result = SquadResult(unique_id, start_logits, end_logits)
        all_results.append(result)

evalTime = timeit.default_timer() - start_time
print("  Evaluation done in total %f secs (%f sec per example)", evalTime, evalTime / len(dataset))

predictions = compute_predictions_logits(
    examples,
    features,
    all_results,
    n_best_size = 20,
    max_answer_length = 30,
    do_lower_case=False,
    output_prediction_file="predictions.json",
    output_nbest_file="nbest_predictions.json",
    output_null_log_odds_file=None,
    verbose_logging=False,
    version_2_with_negative=False,
    null_score_diff_threshold=0.0,
)

# Compute the F1 and exact scores.
results = squad_evaluate(examples, predictions)

print("Results: {}".format(results))

Loading features from cached file %s cached_dev
***** Running evaluation *****
  Num examples =  10970


  Evaluation done in total %f secs (%f sec per example) 325.7774384559998 0.029697122922151303
Results: OrderedDict([('exact', 79.75402081362347), ('f1', 87.70043473208098), ('total', 10570), ('HasAns_exact', 79.75402081362347), ('HasAns_f1', 87.70043473208098), ('HasAns_total', 10570), ('best_exact', 79.75402081362347), ('best_exact_thresh', 0.0), ('best_f1', 87.70043473208098), ('best_f1_thresh', 0.0)])


In [0]:
def predict(q,doc):
    indexed_tokens = tokenizer.encode(q,doc)
    seg_idx = indexed_tokens.index(102)+1
    attention_mask = [1]*len(indexed_tokens)
    segment_ids = [0]*seg_idx+[1]*(len(indexed_tokens)-seg_idx)
    indexed_tokens += [0]*(max_seq_length-len(indexed_tokens))
    attention_mask += [0]*(max_seq_length-len(attention_mask))
    segment_ids += [0]*(max_seq_length-len(segment_ids))
    
    

    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segment_tensor = torch.tensor([segment_ids]).to(device)
    attention_tensor = torch.tensor([attention_mask]).to(device)

    # Predict the start and end positions logits
    with torch.no_grad():
        start_logits, end_logits = model(tokens_tensor, token_type_ids=segment_tensor, attention_mask=attention_tensor)

    # get the highest prediction
    answer = tokenizer.decode(indexed_tokens[torch.argmax(start_logits):torch.argmax(end_logits)+1])
    return answer

In [9]:
doc = "Emmanuel is heading to the bus station"
q = "Where is Emmanuel going to?"
ans = predict(q,doc)
print(ans)

NameError: ignored

In [0]:
doc = "Once upon a time there was an old mother pig who had three little pigs and not enough food to feed them. So when they were old enough, she sent them out into the world to seek their fortunes. The first little pig was very lazy. He didn't want to work at all and he built his house out of straw. The second little pig worked a little bit harder but he was somewhat lazy too and he built his house out of sticks. Then, they sang and danced and played together the rest of the day. The third little pig worked hard all day and built his house with bricks. It was a sturdy house complete with a fine fireplace and chimney. It looked like it could withstand the strongest winds. The next day, a wolf happened to pass by the lane where the three little pigs lived; and he saw the straw house, and he smelled the pig inside. He thought the pig would make a mighty fine meal and his mouth began to water."
q = 'What did the pigs do for the rest of the day?'
ans = predict(q,doc)
print(ans)

sang and danced and played together
